Solving questions from: [2022 hw - week 9](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2022/09-serverless/homework.md)

# Section 1

In this homework, we'll deploy the dino or dragon model we trained in the 
[previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/08-deep-learning/homework.md).

Download the model from here: 

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5


In [1]:
! mkdir models

In [2]:
import requests

url = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5'
response = requests.get(url)

if response.status_code == 200:
    with open('models/dino_dragon_10_0.899.h5', 'wb') as f:
        f.write(response.content)


# Section 2


In [3]:
import tensorflow as tf

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

In [5]:
# loads model
model = tf.keras.models.load_model('models/dino_dragon_10_0.899.h5')

# converts model to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# saves model
with open('models/dino_dragon_10_0.899.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\devsn\AppData\Local\Temp\tmpydih2m_l\assets


INFO:tensorflow:Assets written to: C:\Users\devsn\AppData\Local\Temp\tmpydih2m_l\assets


In [6]:
! dir models

 Volume in drive C is OS
 Volume Serial Number is 34C9-4CBD

 Directory of C:\Users\devsn\Desktop\courses\ML-Zoomcamp\mle\notebooks\2022-hw\models

15-09-2023  02:00    <DIR>          .
15-09-2023  02:00    <DIR>          ..
15-09-2023  01:42        89,759,168 dino_dragon_10_0.899.h5
15-09-2023  02:00        44,866,200 dino_dragon_10_0.899.tflite
               2 File(s)    134,625,368 bytes
               2 Dir(s)  169,039,122,432 bytes free


**Answer**: 43 MB

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

In [8]:
interpreter = tf.lite.Interpreter(model_path='models/dino_dragon_10_0.899.tflite')
interpreter.allocate_tensors()

interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

**Answer**: 13

# Section 3

Preparing the image

In [12]:
import numpy as np

In [9]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

Let's download and resize this image: 

https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

Based on the previous homework, what should be the target size for the image?

*Answer*: (150, 150)

## Question 3

Now we need to turn the image into numpy array and pre-process it.
> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [11]:
img_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
target_size = (150, 150)

raw_img = download_image(img_url)
prepared_img = prepare_image(raw_img, target_size)

type(prepared_img)

PIL.Image.Image

In [15]:
# preprocesses image by dividing by 255
pix = np.array(prepared_img) / 255.

pix.shape

(150, 150, 3)

In [16]:
# returns first element of flattened array
np.ravel(pix)[0]

0.5529411764705883

**Answer**: 0.5529412

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [28]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

X = pix.astype(np.single)
X = np.array([X])
X.shape

(1, 150, 150, 3)

In [26]:
input_index, output_index

(0, 13)

In [29]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

preds

array([[0.82448506]], dtype=float32)

**Answer**: 0.82448614

## Question 5

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

**Answer**: 639 MB

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?


**Answer**: 0.32